# SQL-Challenge
## Setting up SQL-Alchemy and Postgresql:
In the relentless drive to code my way around repetative importing, I have abandoned trying to get my bash script working and am going straight for the sonic screwdriver that is python.

The tables have been created, now i need to bulk load the data into the tables

In [ ]:
import os
import fnmatch
import psycopg2
import psycopg2.extras
import sqlalchemy
import pandas as pd
import postgres_config as pcfg
from sqlalchemy import create_engine

print(pcfg.pg_engine_params)



## Importing tables
The data currently exists as multiple .csv files in a folder somewhere. The next task is to craft a loop with a copy statement that iterates through the csv files. Luckily, the table names are the same as the basename of the files. Don't manualy import your files.
1. declare folder path
2. get list of *.csv
3. loop copy
4. Verify

In [ ]:
# 1-2 Get list of files to import and table names
file_path = 'data/'
tables = [str.split(file,'.')[0] for file in os.listdir(path='data/') if fnmatch.fnmatch(file, '*.csv')]
# Because I had the relationship already defined in the tables, there are foreign key
# dependencies to take into account. I had to re order the lsit accordingly so
# the immaculate tables get loaded first.
# Perhaps develope a way to create an ordered permutation list that ranks the
# name of the tables by number of fk dependencies
permute = [3, 0, 1, 2, 4, 5]
tables =  [tables[i] for i in permute]
print(tables)
print(pcfg.postgres_password)


In [ ]:
#3 loop through list to copy into the tables:
# I googled the copy_expert thing for psycopg2 which lets you write function
# that sends the postgres server a psql copy command. copy is supposed to be very fast.
# I have already defined the schema using an erd tool. this will hopefully
# populate the tables from the csvs. 
#conn = psycopg2.connect(f'dbname={pcfg.postgres_dbname}, user={pcfg.postgres_username}, password={pcfg.postgres_password}, host={pcfg.postgres_address}, port={pcfg.postgres_port}')

#This is really handy. I should turn this into a generic function for use elsewhere.
conn = psycopg2.connect(pcfg.pg_engine_params)
for table_name in tables:
    # we are reading the csv from STDIN. the open command below reads the csv 
    # into stdin
    print(table_name)
    sql = f"COPY {table_name} FROM STDIN WITH CSV HEADER DELIMITER AS ','"
    file = open(f'{file_path}/{table_name}.csv', "r")
    table = table_name
    with conn.cursor() as cur:
        # truncate is a potentialy dangerous function, but we might
        # run this cell several times. This wipes the tables clean so we 
        # don't append repeated data
        cur.execute(f'truncate {table_name} cascade;')
        cur.copy_expert(sql=f'{sql}', file=file)
        conn.commit()
        cur.close()
        
conn.close()

## Creating the database connection
Now it's time to create a bar chart from the view of avg salary by tittle. Way easier than messing with dataframes.

In [ ]:
cnx = create_engine(pcfg.pg_engine_params)
view_name = 'public.average_salary'
avg_salary_df = pd.read_sql(f'SELECT * FROM {view_name}', cnx)
avg_salary_df

In [ ]:
avg_salary_df.plot.bar(x='title', y='salary_average')